In [ ]:
# Подготовка данных для TensorFlow: лог-доходности + экзогенные признаки
import pandas as pd, numpy as np, yfinance as yf
from sklearn.preprocessing import StandardScaler

tf_tickers = ['USDRUB=X','EURRUB=X','BZ=F','GC=F']
START_TF = '2001-01-01'
END_TF   = '2025-12-31'

data_tf = {}
for t in tf_tickers:
    d = yf.download(t, start=START_TF, end=END_TF, progress=False)
    if d.empty or 'Close' not in d:
        continue
    s = d['Close'].copy()
    s.index = pd.to_datetime(s.index)
    idx_b = pd.date_range(s.index.min(), s.index.max(), freq='B')
    s_b = s.reindex(idx_b).ffill()
    data_tf[t] = s_b

df_tf = pd.concat(data_tf, axis=1).dropna()
df_tf.columns = list(data_tf.keys())

# Лог-доходности USDRUB как таргет
df_tf['ret_USDRUB'] = np.log(df_tf['USDRUB=X']/df_tf['USDRUB=X'].shift(1))

# Лог-доходности экзогенных активов
for t in [c for c in df_tf.columns if c in ['EURRUB=X','BZ=F','GC=F']]:
    df_tf[f'ret_{t}'] = np.log(df_tf[t]/df_tf[t].shift(1))

# Техиндикаторы по USDRUB
def rsi_tf(series, n=14):
    delta = series.diff()
    up = delta.clip(lower=0).rolling(n).mean()
    down = (-delta.clip(upper=0)).rolling(n).mean()
    rs = up/(down.replace(0, np.nan))
    return 100 - (100/(1+rs))

ret_usd_full_tf = np.log(df_tf['USDRUB=X']/df_tf['USDRUB=X'].shift(1))
df_tf['usdrub_vol_20'] = ret_usd_full_tf.rolling(20).std()
df_tf['usdrub_ma_5']  = df_tf['USDRUB=X'].rolling(5).mean()
df_tf['usdrub_ma_20'] = df_tf['USDRUB=X'].rolling(20).mean()
df_tf['usdrub_ma_60'] = df_tf['USDRUB=X'].rolling(60).mean()
df_tf['usdrub_rsi_14'] = rsi_tf(df_tf['USDRUB=X'], 14)
roll_max_tf = df_tf['USDRUB=X'].rolling(60).max()
df_tf['usdrub_drawdown_60'] = df_tf['USDRUB=X']/roll_max_tf - 1.0

# Лаги лог-доходностей USDRUB, чтобы модель видела динамику
for lag in [1, 2, 3, 5]:
    df_tf[f'ret_USDRUB_lag{lag}'] = df_tf['ret_USDRUB'].shift(lag)

df_tf = df_tf.dropna()

# Ограничиваемся последними годами, чтобы убрать старые режимы рынка
df_tf = df_tf.loc[df_tf.index >= '2015-01-01']

# Разделяем на train (до конца 2024) и период прогноза 2025
split_date = pd.Timestamp('2024-12-31')
df_train = df_tf.loc[df_tf.index <= split_date].copy()
df_forecast = df_tf.loc[df_tf.index > split_date].copy()

feature_cols = [c for c in df_tf.columns if c not in ['USDRUB=X','ret_USDRUB']]
X_train_raw = df_train[feature_cols].values
y_train = df_train['ret_USDRUB'].values.reshape(-1, 1)
X_forecast_raw = df_forecast[feature_cols].values
y_true_ret_2025 = df_forecast['ret_USDRUB'].values.reshape(-1, 1)

scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_all = scaler_X.fit_transform(np.vstack([X_train_raw, X_forecast_raw]))
X_train_scaled = X_all[:len(X_train_raw)]
X_forecast_scaled = X_all[len(X_train_raw):]
y_train_scaled = scaler_y.fit_transform(y_train)

# Формируем LSTM-окна
window = 30
X_seq = []
y_seq = []
for i in range(window, len(X_train_scaled)):
    X_seq.append(X_train_scaled[i-window:i, :])
    y_seq.append(y_train_scaled[i, 0])
X_seq = np.array(X_seq)
y_seq = np.array(y_seq).reshape(-1, 1)

print('TF LSTM train shape:', X_seq.shape, y_seq.shape)
print('TF forecast (scaled) shape:', X_forecast_scaled.shape, y_true_ret_2025.shape)

In [ ]:
# TensorFlow модель: LSTM по временным окнам
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

tf.random.set_seed(123)

model_tf = keras.Sequential([
    layers.Input(shape=(X_seq.shape[1], X_seq.shape[2])),
    layers.LSTM(64, return_sequences=False),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='linear')
])

model_tf.compile(optimizer=keras.optimizers.Adam(learning_rate=5e-4), loss='mse')

history_tf = model_tf.fit(
    X_seq, y_seq,
    epochs=20,
    batch_size=64,
    validation_split=0.2,
    verbose=0
)

print('Final train loss:', history_tf.history['loss'][-1])
print('Final val loss:', history_tf.history['val_loss'][-1])

In [ ]:
# Вариант 1: краткосрочный прогноз на июнь 2025 года для USDRUB
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Окно прогноза: весь июнь 2025
mask_june = (df_forecast.index >= "2025-06-01") & (df_forecast.index <= "2025-06-30")
df_forecast_short = df_forecast.loc[mask_june].copy()
X_forecast_short_scaled = X_forecast_scaled[mask_june]

print("USDRUB short window dates:", df_forecast_short.index.min(), "->", df_forecast_short.index.max())
print("USDRUB short window length:", len(df_forecast_short))

# Строим начальное окно из последних train-данных
last_train_scaled = X_train_scaled[-window:, :]
prices_ret_scaled = []
for i in range(len(X_forecast_short_scaled)):
    x_win = last_train_scaled if i == 0 else np.vstack([x_win[1:], X_forecast_short_scaled[i:i+1]])
    pred_scaled = model_tf.predict(x_win[np.newaxis, ...], verbose=0)[0, 0]
    prices_ret_scaled.append(pred_scaled)

y_pred_ret_short_scaled = np.array(prices_ret_scaled).reshape(-1, 1)
y_pred_ret_short = scaler_y.inverse_transform(y_pred_ret_short_scaled).reshape(-1)

# Ограничиваем экстремальные доходности и добавляем небольшой дрейф
y_pred_ret_short = np.clip(y_pred_ret_short, -0.05, 0.05)
drift = 0.0005  # ~0.05% в день
y_pred_ret_short = y_pred_ret_short + drift

# Восстанавливаем цену: стартуем с реальной цены на split_date (последний train-день)
P_anchor = float(df_train['USDRUB=X'].iloc[-1])
prices_tf = [P_anchor]
for rt in y_pred_ret_short:
    prices_tf.append(prices_tf[-1] * np.exp(rt))

prices_tf_series = pd.Series(prices_tf[1:], index=df_forecast_short.index)
actual_short = df_forecast_short['USDRUB=X']

plt.figure(figsize=(14, 6))
plt.plot(actual_short.index, actual_short.values, label='Actual June 2025')
plt.plot(prices_tf_series.index, prices_tf_series.values, label='TF LSTM forecast June 2025', linestyle='--')
plt.title('USDRUB — TensorFlow LSTM, июнь 2025')
plt.xlabel('Дата')
plt.ylabel('Курс')
plt.legend()
plt.grid(True)
plt.show()

# Метрики и корреляция на этом окне
common_tf = actual_short.index.intersection(prices_tf_series.index)
y_true_tf = actual_short.loc[common_tf].to_numpy(dtype=float)
y_pred_tf = prices_tf_series.loc[common_tf].to_numpy(dtype=float)
N_tf = min(len(y_true_tf), len(y_pred_tf))
y_true_tf, y_pred_tf = y_true_tf[:N_tf], y_pred_tf[:N_tf]
mae_tf = float(np.mean(np.abs(y_pred_tf - y_true_tf))) if N_tf > 0 else np.nan
rmse_tf = float(np.sqrt(np.mean((y_pred_tf - y_true_tf) ** 2))) if N_tf > 0 else np.nan
mape_tf = float(np.mean(np.abs((y_pred_tf - y_true_tf) / np.maximum(1e-8, y_true_tf)))) * 100 if N_tf > 0 else np.nan
corr_tf = float(pd.Series(y_true_tf).corr(pd.Series(y_pred_tf))) if N_tf > 2 else np.nan
print(f'USDRUB June 2025: MAE={mae_tf:.4f} RMSE={rmse_tf:.4f} MAPE={mape_tf:.2f}% Corr={corr_tf:.3f}')

In [ ]:
# Вариант 2: веер сценариев на N рабочих дней вперёд
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

# Те же данные окна, что и в варианте 1
N_DAYS_AHEAD = 15
start_idx = 0
end_idx = min(N_DAYS_AHEAD, len(df_forecast))
df_forecast_short = df_forecast.iloc[start_idx:end_idx].copy()
X_forecast_short_scaled = X_forecast_scaled[start_idx:end_idx]

# Количество сценариев
N_SCENARIOS = 5

scenario_prices = []
scenario_labels = []

for s in range(N_SCENARIOS):
    # Для разнообразия немного меняем сид
    tf.random.set_seed(123 + s)

    last_train_scaled = X_train_scaled[-window:, :]
    prices_ret_scaled = []
    for i in range(len(X_forecast_short_scaled)):
        x_win = last_train_scaled if i == 0 else np.vstack([x_win[1:], X_forecast_short_scaled[i:i+1]])
        pred_scaled = model_tf.predict(x_win[np.newaxis, ...], verbose=0)[0, 0]
        prices_ret_scaled.append(pred_scaled)

    y_pred_scaled = np.array(prices_ret_scaled).reshape(-1, 1)
    y_pred = scaler_y.inverse_transform(y_pred_scaled).reshape(-1)
    y_pred = np.clip(y_pred, -0.05, 0.05)
    drift = 0.0005
    y_pred = y_pred + drift

    P_anchor = float(df_train['USDRUB=X'].iloc[-1])
    prices_tf = [P_anchor]
    for rt in y_pred:
        prices_tf.append(prices_tf[-1] * np.exp(rt))
    scenario_prices.append(pd.Series(prices_tf[1:], index=df_forecast_short.index))
    scenario_labels.append(f'Scenario {s+1}')

actual_short = df_forecast_short['USDRUB=X']

plt.figure(figsize=(14, 6))
plt.plot(actual_short.index, actual_short.values, label='Actual short window', linewidth=2, color='black')
for series, label in zip(scenario_prices, scenario_labels):
    plt.plot(series.index, series.values, linestyle='--', alpha=0.7, label=label)
plt.title(f'USDRUB — LSTM сценарный веер, {N_DAYS_AHEAD} рабочих дней вперёд')
plt.xlabel('Дата')
plt.ylabel('Курс')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# =======================
# AAPL: LSTM краткосрочный прогноз и сценарный веер
# =======================

# 1. Подготовка данных для AAPL
import pandas as pd, numpy as np, yfinance as yf
from sklearn.preprocessing import StandardScaler

aapl_tickers = ['AAPL', 'SPY', 'GC=F']  # цель + пара простых экзогенов
START_AAPL = '2010-01-01'
END_AAPL   = '2025-12-31'

data_aapl = {}
for t in aapl_tickers:
    d = yf.download(t, start=START_AAPL, end=END_AAPL, progress=False)
    if d.empty or 'Close' not in d:
        continue
    s = d['Close'].copy()
    s.index = pd.to_datetime(s.index)
    idx_b = pd.date_range(s.index.min(), s.index.max(), freq='B')
    s_b = s.reindex(idx_b).ffill()
    data_aapl[t] = s_b

df_aapl = pd.concat(data_aapl, axis=1).dropna()
df_aapl.columns = list(data_aapl.keys())

# Лог-доходности AAPL как таргет
df_aapl['ret_AAPL'] = np.log(df_aapl['AAPL']/df_aapl['AAPL'].shift(1))

# Лог-доходности экзогенов
for t in [c for c in df_aapl.columns if c in ['SPY','GC=F']]:
    df_aapl[f'ret_{t}'] = np.log(df_aapl[t]/df_aapl[t].shift(1))

# Пара простых техиндикаторов по AAPL
ret_aapl_full = np.log(df_aapl['AAPL']/df_aapl['AAPL'].shift(1))
df_aapl['aapl_vol_20'] = ret_aapl_full.rolling(20).std()
df_aapl['aapl_ma_5']  = df_aapl['AAPL'].rolling(5).mean()
df_aapl['aapl_ma_20'] = df_aapl['AAPL'].rolling(20).mean()

# Лаги лог-доходностей AAPL
for lag in [1, 2, 3, 5]:
    df_aapl[f'ret_AAPL_lag{lag}'] = df_aapl['ret_AAPL'].shift(lag)

df_aapl = df_aapl.dropna()

# Разделяем на train (до конца 2024) и период прогноза 2025
split_date_aapl = pd.Timestamp('2024-12-31')
df_train_aapl = df_aapl.loc[df_aapl.index <= split_date_aapl].copy()
df_forecast_aapl = df_aapl.loc[df_aapl.index > split_date_aapl].copy()

feature_cols_aapl = [c for c in df_aapl.columns if c not in ['AAPL','ret_AAPL']]
X_train_aapl_raw = df_train_aapl[feature_cols_aapl].values
y_train_aapl = df_train_aapl['ret_AAPL'].values.reshape(-1, 1)
X_forecast_aapl_raw = df_forecast_aapl[feature_cols_aapl].values

scaler_X_aapl = StandardScaler()
scaler_y_aapl = StandardScaler()
X_all_aapl = scaler_X_aapl.fit_transform(np.vstack([X_train_aapl_raw, X_forecast_aapl_raw]))
X_train_aapl_scaled = X_all_aapl[:len(X_train_aapl_raw)]
X_forecast_aapl_scaled = X_all_aapl[len(X_train_aapl_raw):]
y_train_aapl_scaled = scaler_y_aapl.fit_transform(y_train_aapl)

# Формируем LSTM-окна для AAPL
window_aapl = 30
X_seq_aapl = []
y_seq_aapl = []
for i in range(window_aapl, len(X_train_aapl_scaled)):
    X_seq_aapl.append(X_train_aapl_scaled[i-window_aapl:i, :])
    y_seq_aapl.append(y_train_aapl_scaled[i, 0])
X_seq_aapl = np.array(X_seq_aapl)
y_seq_aapl = np.array(y_seq_aapl).reshape(-1, 1)

print('AAPL LSTM train shape:', X_seq_aapl.shape, y_seq_aapl.shape)

# 2. Модель LSTM для AAPL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

tf.random.set_seed(321)

model_aapl = keras.Sequential([
    layers.Input(shape=(X_seq_aapl.shape[1], X_seq_aapl.shape[2])),
    layers.LSTM(64, return_sequences=False),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='linear')
])

model_aapl.compile(optimizer=keras.optimizers.Adam(learning_rate=5e-4), loss='mse')

history_aapl = model_aapl.fit(
    X_seq_aapl, y_seq_aapl,
    epochs=30,
    batch_size=64,
    validation_split=0.2,
    verbose=0
)

print('AAPL train loss:', history_aapl.history['loss'][-1])
print('AAPL val loss:', history_aapl.history['val_loss'][-1])

# 3. Вариант 1: краткосрочный прогноз AAPL на N_DAYS_AHEAD
N_DAYS_AHEAD_AAPL = 15
start_idx_aapl = 0
end_idx_aapl = min(N_DAYS_AHEAD_AAPL, len(df_forecast_aapl))
df_forecast_aapl_short = df_forecast_aapl.iloc[start_idx_aapl:end_idx_aapl].copy()
X_forecast_aapl_short_scaled = X_forecast_aapl_scaled[start_idx_aapl:end_idx_aapl]

last_train_aapl_scaled = X_train_aapl_scaled[-window_aapl:, :]
prices_ret_aapl_scaled = []
for i in range(len(X_forecast_aapl_short_scaled)):
    x_win = last_train_aapl_scaled if i == 0 else np.vstack([x_win[1:], X_forecast_aapl_short_scaled[i:i+1]])
    pred_scaled = model_aapl.predict(x_win[np.newaxis, ...], verbose=0)[0, 0]
    prices_ret_aapl_scaled.append(pred_scaled)

y_pred_ret_aapl_short_scaled = np.array(prices_ret_aapl_scaled).reshape(-1, 1)
y_pred_ret_aapl_short = scaler_y_aapl.inverse_transform(y_pred_ret_aapl_short_scaled).reshape(-1)

# Ограничиваем экстремальные доходности и добавляем небольшой дрейф для одиночного прогноза
y_pred_ret_aapl_short_single = np.clip(y_pred_ret_aapl_short, -0.05, 0.05)
drift_aapl_single = 0.0005
y_pred_ret_aapl_short_single = y_pred_ret_aapl_short_single + drift_aapl_single

P_anchor_aapl = float(df_train_aapl['AAPL'].iloc[-1])
prices_aapl = [P_anchor_aapl]
for rt in y_pred_ret_aapl_short_single:
    prices_aapl.append(prices_aapl[-1] * np.exp(rt))

prices_aapl_series = pd.Series(prices_aapl[1:], index=df_forecast_aapl_short.index)
actual_aapl_short = df_forecast_aapl_short['AAPL']

plt.figure(figsize=(14, 6))
plt.plot(actual_aapl_short.index, actual_aapl_short.values, label='AAPL actual short window')
plt.plot(prices_aapl_series.index, prices_aapl_series.values, label=f'AAPL LSTM forecast {N_DAYS_AHEAD_AAPL} days', linestyle='--')
plt.title(f'AAPL — TensorFlow LSTM, {N_DAYS_AHEAD_AAPL} рабочих дней вперёд')
plt.xlabel('Дата')
plt.ylabel('Цена AAPL')
plt.legend()
plt.grid(True)
plt.show()